In [1]:
import pandas as pd
import numpy as np
from ofxparse import OfxParser
import matplotlib.pyplot as plt
import os
import tabula
from datetime import datetime

In [2]:
# get downloaded statments
def get_files(filetype):
    files = []
    for f in os.listdir('statements/'):
        if not f.startswith('.') and ("." + filetype) in f: 
            files.append("statements/" + f)
    
    return files

In [3]:
# returnt he transaction from .ofx file type
def get_transactions(filename):
    with open(filename) as f:
        ofx = OfxParser.parse(f)

    account = ofx.account
    statement = account.statement
    transactions = statement.transactions

    return transactions

In [4]:
# build dataframe
df = pd.DataFrame(columns=['date', 'year', 'month', 'day', 'description', 'debit', 'credit'])

files = get_files("ofx")

for f in files:
    for t in get_transactions(f):
        debit = t.amount if t.type == "debit" else 0
        credit = t.amount if t.type == "credit" else 0
        new = {'date':t.date, 'year': t.date.year, 'month': t.date.month, 'day': t.date.day, 'description': t.memo, 'debit': debit, 'credit': credit}
        
        df = df.append(new, ignore_index=True)

df = df.astype({"description": str, "debit": float, "credit": float})

In [50]:
df.to_pickle("transactions.pkl")